(3 pts) What is a neural network? What are the general steps required to build a neural network?              
(3 pts) Generally, how do you check the performance of a neural network? Why is this the case?               
(4 pts) Clean the data or do additional cleaning if you have used the dataset for another assignment. Specify the improvements (at least 2) that you made to your cleaning if you selected the dataset before. If you select one with a low number or records, consider oversampling.                    
(10 pts) Create a neural network using Keras or PyTorch to predict the outcome of your datasets.           
(5 pts) Compare the performance of the neural networks to another model you created. Which performed better? Why do you think that is?                       

# What is a neural network

A neural network is a way to diagram relationships between data in a way that is mapped like the human brain. The general steps are :
1) obtain and preprocess the data
2) Set the paramaters of the network
3) Train the network by feeding it the data
4) Fine tune as needed

# How to check performance and why? 

You can check its performance by doing a classification matrix and calculating the recall, precision, accuracy, and any other optimization metric. This is the case because it allows us to see how well the machine is able to predict the trends by putting what it predicts against what the actual value is. 

# Cleaning the Data - credit card set

In [1]:
import pandas as pd
import numpy as np
card_df = pd.read_csv('Credit_card.csv')
label_df = pd.read_csv('Credit_card_label.csv')

credit_df = pd.merge(card_df,label_df)

credit_df = pd.get_dummies(credit_df)

credit_df.fillna(credit_df.mean(), inplace=True)




/var/folders/68/jgh30zmn54z999vhrzlb3zbc0000gn/T/ipykernel_18370/4155478247.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
